In [6]:
import cv2
import os


In [8]:
def mouseCrop(event, x, y, flags, param):
    global x_start, y_start, x_end, y_end, cropping

    if event == cv2.EVENT_LBUTTONDOWN:
        x_start, y_start, x_end, y_end = x, y, x, y
        cropping = True

    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping == True:
            x_end, y_end = x, y

    elif event == cv2.EVENT_LBUTTONUP:
        x_end, y_end = x, y
        cropping = False


def changePic(direction):
    global img_num, x_start, y_start, x_end, y_end

    cv2.destroyAllWindows()
    x_start, y_start, x_end, y_end = 0, 0, 0, 0
    if direction == 'next':
        img_num += 1
    elif direction == 'previous':
        img_num -= 1


In [10]:
#Creating Folders
if not os.path.exists('positives'):
    os.makedirs('positives')

if not os.path.exists('negatives'):
    os.makedirs('negatives')


x_start, y_start, x_end, y_end = 0, 0, 0, 0
cropping = False


window_name = '1P, 2N'
img_num = 0
while True:
    # Looping through all images
    images = []
    for file in os.listdir('images'):
        images.append(file)
    if len(images) == 0:
        print('No pictures in Images folder')
        break
    img_name = images[img_num]

    #Current Positives
    positives = []
    for pos_file in os.listdir('positives'):
        positives.append(int(pos_file[2:-4]))
    last_positive = max(positives) if len(positives) > 0 else 0

    #Current Negatives
    negatives = []
    for neg_file in os.listdir('negatives'):
        negatives.append(int(neg_file[2:-4]))
    last_negative = max(negatives) if len(negatives) > 0 else 0

    #reading image
    img = cv2.imread('images/' + img_name)
    #img = cv2.resize(img, (1000, 1000))

    #Showing Rectangle Selection
    cv2.setMouseCallback(window_name, mouseCrop)
    cv2.rectangle(img, (x_start, y_start), (x_end, y_end), (255, 0, 0), 2)

    #Showing Images
    cv2.imshow(window_name, img)
    k = cv2.waitKey(1)

    if k == ord('q'):
        break

    elif k == ord('d'):
        if img_num < len(images)-1:
            changePic('next')

    elif k == ord('a'):
        if img_num > 0:
            changePic('previous')

    # saves cropped images in positive and moves original to categorized
    #1 for pos, 2 for neg, 3 for none
    elif k in [ord('1'), ord('2'), ord('3')]:
        x = [x_start, x_end]
        y = [y_start, y_end]

        x_start = min(x)
        x_end = max(x)
        y_start = min(y)
        y_end = max(y)

        imgCrop = img[y_start + 2:y_end - 1, x_start + 2:x_end - 1]
        if k == ord('1'):
            cv2.imwrite('positives/p_' + str(last_positive + 1) + '.jpg', imgCrop)
        elif k == ord('2'):
            cv2.imwrite('negatives/n_' + str(last_negative + 1) + '.jpg', imgCrop)
        changePic('next')
        os.rename('images/' + img_name, 'categorized/' + img_name)



error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
